In [121]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import string
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
import yaml
import ast

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier, 
                              HistGradientBoostingClassifier, StackingClassifier, VotingClassifier)
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import (balanced_accuracy_score, f1_score,
                             accuracy_score, confusion_matrix)

from sklearn.model_selection import train_test_split

from src.constants import *
from src.utils import load_yaml
os.chdir('f:\\iNeuron\\Projects\\Scania_Truck_Failures')

## Loading Params.yaml

In [2]:
params_config = load_yaml(PARAMS_PATH)
params_config

[2024-01-12 07:57:25,834: INFO: utils: params.yaml yaml_file is loaded]


ConfigBox({'optuna': {'Logistic_Regression': {'penalty': "trial.suggest_categorical('penalty', ['l2', None])"}, 'SGD_Classifier': {'loss': "trial.suggest_categorical('loss', ['squared_epsilon_insensitive', 'epsilon_insensitive', 'huber', 'squared_error', 'perceptron', 'squared_hinge', 'hinge', 'log_loss', 'modified_huber'])"}, 'Random Forest': {'n_estimators': "trial.suggest_int('n_estimators', 100, 1500)", 'criterion': "trial.suggest_categorical('criterion', ['log_loss', 'entropy', 'gini'])", 'max_features': "trial.suggest_categorical('max_features', ['sqrt', 'log2', None])", 'class_weight': "trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample'])"}, 'Ada_Boost': {'n_estimators': "trial.suggest_int('n_estimators', 100, 1500)", 'algorithm': "trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R'])"}, 'Grad_Boost': {'loss': "trial.suggest_categorical('loss', ['log_loss', 'exponential'])", 'n_estimators': "trial.suggest_int('n_estimators', 100, 1500)", 'criterio

In [ ]:
dict(params_config.randomized_search_cv['Random Forest'])

## HyperParameter Tuning

In [3]:
with open('params.yaml', 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)

In [ ]:
config['optuna']['Logistic_Regression']
space = {}
for key,value in config['optuna']['Random Forest'].items():
    space[key] = eval(value)

In [ ]:
config['optuna']['Logistic_Regression']['penalty']

In [ ]:
list(config['optuna']['Logistic_Regression'].keys())[0]

In [ ]:
config['optuna']['Logistic_Regression']['penalty']

In [120]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
train_df = pd.read_csv(".//artifacts\data\processed\stage_2_processing\processed_train_data.csv")
test_df = pd.read_csv(".//artifacts\data\processed\stage_2_processing\processed_test_data.csv")

x_train = train_df.drop(columns='class')
y_train = train_df['class']
x_test = test_df.drop(columns='class')
y_test = test_df['class']
print(train_df.shape,test_df.shape)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(1978, 171) (1970, 171)
(1978, 170) (1970, 170) (1978,) (1970,)


### Optuna

In [123]:
import optuna
def objective(trial,train_x = x_train,test_x = x_test, y_train = y_train, y_test = y_test):
  # train_x,test_x,y_train,y_test=train_test_split(data,target,test_size=0.20,random_state=25)
  param= {list(config['optuna']['Logistic_Regression'].keys())[0] : eval(config['optuna']['Logistic_Regression']['penalty'])}
    
  log_reg=LogisticRegression(**param)
  log_reg.fit(train_x,y_train)
  y_predict=log_reg.predict(test_x)
  accuracy_score_=accuracy_score(y_test,y_predict)
  return accuracy_score_
find_param=optuna.create_study(direction = "maximize")
find_param.optimize(objective,n_trials=10)

[I 2024-01-12 20:35:05,870] A new study created in memory with name: no-name-0b3f0a3a-3620-4de9-82f6-f20741e8947f


[I 2024-01-12 20:35:06,223] Trial 0 finished with value: 0.4964467005076142 and parameters: {'penalty': None}. Best is trial 0 with value: 0.4964467005076142.
[I 2024-01-12 20:35:06,429] Trial 1 finished with value: 0.49086294416243653 and parameters: {'penalty': 'l2'}. Best is trial 0 with value: 0.4964467005076142.
[I 2024-01-12 20:35:06,638] Trial 2 finished with value: 0.49086294416243653 and parameters: {'penalty': 'l2'}. Best is trial 0 with value: 0.4964467005076142.
[I 2024-01-12 20:35:06,909] Trial 3 finished with value: 0.4964467005076142 and parameters: {'penalty': None}. Best is trial 0 with value: 0.4964467005076142.
[I 2024-01-12 20:35:07,114] Trial 4 finished with value: 0.4964467005076142 and parameters: {'penalty': None}. Best is trial 0 with value: 0.4964467005076142.
[I 2024-01-12 20:35:07,303] Trial 5 finished with value: 0.49086294416243653 and parameters: {'penalty': 'l2'}. Best is trial 0 with value: 0.4964467005076142.
[I 2024-01-12 20:35:07,393] Trial 6 finishe

In [ ]:
find_param.best_trial.value

In [154]:
import json
eval('find_param.best_trial.params')

{'penalty': None}

In [153]:
find_param.best_trial.params

{'penalty': None}

In [ ]:
find_param.trials_dataframe()

### HyperOpt

In [ ]:
from hyperopt.pyll.base import scope
import hyperopt
a = scope.int(hp.quniform('n_estimators', 100, 2000, 1))

print(hyperopt.pyll.stochastic.sample(a))


In [ ]:
config['hyperopt']['Random Forest']

In [ ]:
config['hyperopt']['Random Forest'].items()

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval
from hyperopt.pyll.base import scope
space = {}
for key,value in config['hyperopt']['Random Forest'].items():
    space[key] = eval(value)
def objective(space):
    hp_rand_forest = RandomForestClassifier(**space)
    hp_rand_forest.fit(train_x,y_train)
    y_pred = hp_rand_forest.predict(test_x)
    tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
    cost = float((10*fp)+(500*fn))
    return cost

trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 10,
            trials= trials)
best_randf=space_eval(space,best)
best_randf

In [ ]:
trials.average_best_error()

In [ ]:
best_randf

In [ ]:
b = {'n_estimators': 352,
 'criterion': 'entropy',
 'max_features': 'log2',
 'class_weight': 'balanced_subsample'}
randf=RandomForestClassifier(**b)
randf.fit(train_x,y_train)
y_pred=randf.predict(test_x)
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
cost = float((10*fp)+(500*fn))
print(cost)

### RandomSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
model = XGBClassifier(njobs = -1)
model_name = 'XGB_Classifier'
params_config = load_yaml(PARAMS_PATH)

# def cost_scorer(estimator, X, y):
#     x_train_, x_test_, y_train_, y_test_ = train_test_split(X,y, random_state=8, test_size = 0.25)
#     estimator.fit(x_train_,y_train_)
#     y_pred = estimator.predict(x_test_)
#     tn, fp, fn, tp = confusion_matrix(y_true=y_test_, y_pred=y_pred).ravel()
#     cost = float((10*fp)+(500*fn))
#     return cost

params_grid = dict(params_config.randomized_search_cv[model_name])
random_search_cv = RandomizedSearchCV(estimator = model,
                                        param_distributions = params_grid,
                                        n_iter = 5,
                                        scoring = 'accuracy',
                                        n_jobs = -1,
                                        verbose = 3,
                                        random_state = 8
                                        )
random_search_cv.fit(x_train,y_train)

In [ ]:
random_search_cv.best_score_

In [ ]:
random_search_cv.best_params_

In [ ]:
model = RandomForestClassifier(**random_search_cv.best_params_)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
cost = float((10*fp)+(500*fn))
print(cost)

### Using the User Defined Function: hyper_parameter_tuning()
This is done to call optuna and hyperopt at the same time

In [ ]:
type(LogisticRegression())

In [3]:
from src.utils import parameter_tuning
report = parameter_tuning(model_class = SGDClassifier,
                          model_name = 'SGD_Classifier',
                          x_train = x_train,
                          x_test = x_test,
                          y_train = y_train,
                          y_test = y_test)

[2024-01-12 00:03:35,963: INFO: utils: params.yaml yaml_file is loaded]


[I 2024-01-12 00:03:35,966] A new study created in memory with name: no-name-3a2364cf-7cfe-4ace-bff1-9ec28564898f
[I 2024-01-12 00:03:36,029] Trial 0 finished with value: 279830.0 and parameters: {'loss': 'modified_huber'}. Best is trial 0 with value: 279830.0.
[I 2024-01-12 00:03:36,058] Trial 1 finished with value: 189640.0 and parameters: {'loss': 'squared_hinge'}. Best is trial 1 with value: 189640.0.


[I 2024-01-12 00:03:36,111] Trial 2 finished with value: 225500.0 and parameters: {'loss': 'log_loss'}. Best is trial 1 with value: 189640.0.
[I 2024-01-12 00:03:36,161] Trial 3 finished with value: 167180.0 and parameters: {'loss': 'epsilon_insensitive'}. Best is trial 3 with value: 167180.0.
[I 2024-01-12 00:03:36,231] Trial 4 finished with value: 147690.0 and parameters: {'loss': 'squared_error'}. Best is trial 4 with value: 147690.0.
[I 2024-01-12 00:03:36,383] Trial 5 finished with value: 109750.0 and parameters: {'loss': 'squared_epsilon_insensitive'}. Best is trial 5 with value: 109750.0.
[I 2024-01-12 00:03:36,410] Trial 6 finished with value: 16960.0 and parameters: {'loss': 'squared_hinge'}. Best is trial 6 with value: 16960.0.
[I 2024-01-12 00:03:36,448] Trial 7 finished with value: 128870.0 and parameters: {'loss': 'perceptron'}. Best is trial 6 with value: 16960.0.
[I 2024-01-12 00:03:36,536] Trial 8 finished with value: 439620.0 and parameters: {'loss': 'squared_error'}. 

Optuna: SGD_Classifier --- {'cost': 16960.0, 'params': {'loss': 'squared_hinge'}}


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?][2024-01-12 00:03:36,597: INFO: tpe: build_posterior_wrapper took 0.003051 seconds]
[2024-01-12 00:03:36,599: INFO: tpe: TPE using 0 trials]
[2024-01-12 00:03:36,668: INFO: tpe: build_posterior_wrapper took 0.000998 seconds]
[2024-01-12 00:03:36,671: INFO: tpe: TPE using 1/1 trials with best loss 310900.000000]
 20%|██        | 2/10 [00:00<00:00,  9.92trial/s, best loss: 310900.0][2024-01-12 00:03:36,799: INFO: tpe: build_posterior_wrapper took 0.001052 seconds]
[2024-01-12 00:03:36,801: INFO: tpe: TPE using 2/2 trials with best loss 310900.000000]
 30%|███       | 3/10 [00:00<00:00,  9.12trial/s, best loss: 289000.0][2024-01-12 00:03:36,919: INFO: tpe: build_posterior_wrapper took 0.000000 seconds]
[2024-01-12 00:03:36,922: INFO: tpe: TPE using 3/3 trials with best loss 289000.000000]
[2024-01-12 00:03:36,980: INFO: tpe: build_posterior_wrapper took

In [4]:
report

{'Optuna': {'cost': 16960.0, 'params': {'loss': 'squared_hinge'}},
 'HyperOpt': {'cost': 127890.0, 'params': {'loss': 'log_loss'}}}

In [10]:
report = {'Logistic_Regression': {'Optuna': {'cost': 437200.0, 'params': {'penalty': None}}, 'HyperOpt': {'cost': 437200.0, 'params': {'penalty': None}}, 'Best_Params': {'penalty': None}, 'Cost': 437200.0}, 'SGD_Classifier': {'Optuna': {'cost': 31760.0, 'params': {'loss': 'squared_error'}}, 'HyperOpt': {'cost': 4840.0, 'params': {'loss': 'squared_error'}}, 'Best_Params': {'loss': 'squared_error'}, 'Cost': 4840.0}}
costs = [value['Cost'] for value in report.values()]
min_cost = min(costs)

In [15]:
report.values()

dict_values([{'Optuna': {'cost': 437200.0, 'params': {'penalty': None}}, 'HyperOpt': {'cost': 437200.0, 'params': {'penalty': None}}, 'Best_Params': {'penalty': None}, 'Cost': 437200.0}, {'Optuna': {'cost': 31760.0, 'params': {'loss': 'squared_error'}}, 'HyperOpt': {'cost': 4840.0, 'params': {'loss': 'squared_error'}}, 'Best_Params': {'loss': 'squared_error'}, 'Cost': 4840.0}])

In [11]:
min_cost

4840.0

In [5]:
min_value = min(report['Optuna']['cost'],report['HyperOpt']['cost'])
if min_value == report['Optuna']['cost']:
    params = report['Optuna']['params']
else:
    params = report['HyperOpt']['params']

In [7]:
min_value

16960.0

In [6]:
params

{'loss': 'squared_hinge'}

In [6]:
dictionary = {'Logistic_Regression': {'Optuna': {'cost': 437200.0, 'params': {'penalty': None}}, 'HyperOpt': {'cost': 437200.0, 'params': {'penalty': None}}, 'Best_Params': {'penalty': None}, 'Best_Cost': 437200.0}, 'SGD_Classifier': {'Optuna': {'cost': 125050.0, 'params': {'loss': 'log_loss'}}, 'HyperOpt': 
{'cost': 15060.0, 'params': {'loss': 'hinge'}}, 'Best_Params': {'loss': 'hinge'}, 'Best_Cost': 15060.0}}

In [82]:
dictionary

{'Logistic_Regression': {'Optuna': {'cost': 437200.0,
   'params': {'penalty': None}},
  'HyperOpt': {'cost': 437200.0, 'params': {'penalty': None}},
  'Best_Params': {'penalty': None},
  'Best_Cost': 437200.0},
 'SGD_Classifier': {'Optuna': {'cost': 125050.0,
   'params': {'loss': 'log_loss'}},
  'HyperOpt': {'cost': 15060.0, 'params': {'loss': 'hinge'}},
  'Best_Params': {'loss': 'hinge'},
  'Best_Cost': 15060.0}}

In [83]:
len(dictionary)

2

In [13]:
dictionary['SGD_Classifier'].keys()
costs = [value['Best_Cost'] for value in dictionary.values()]
min_cost = min(costs)
min_cost

15060.0

In [31]:
models = {'Logistic_Regression': LogisticRegression(), 
            'SGD_Classifier': SGDClassifier(),
            'Random Forest': RandomForestClassifier, 
            'Ada_Boost': AdaBoostClassifier, 
            'Grad_Boost': GradientBoostingClassifier, 
            'Bagging_Classifier': BaggingClassifier, 
            'ExtraTreesClassifier': ExtraTreesClassifier, 
            'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
            'Decision_Tree_Classifier': DecisionTreeClassifier,
            'XGB_Classifier': XGBClassifier,
            'KNN_Classifier': KNeighborsClassifier,
            'MLP_Classifier': MLPClassifier
            }
for i in dictionary.keys():
    if min_cost == dictionary[i]['Best_Cost']:
        best_params_so_far = dictionary[i]['Best_Params']
print(best_params_so_far)


##Same code as a list comprehension
best_params_so_far_ = [(i, min_cost, dictionary[i]['Best_Params']) for i in dictionary.keys() if min_cost == dictionary[i]['Best_Cost']]
print(best_params_so_far_[0])

model = models[best_params_so_far_[0][0]]
model(**best_params_so_far_[0][2])

{'loss': 'hinge'}
('SGD_Classifier', 15060.0, {'loss': 'hinge'})


SGDClassifier()

##### Fetching the dataframe of 1st five best models for Stacking Classifier

In [46]:
best_models_ = sorted(dictionary.items(), key = lambda x: x[1]['Best_Cost'])[:5]
best_models = [best_models_[i][0] for i in range(len(best_models_))]
best_models_with_params = []
for i in best_models:
    best_models_with_params.append((i,dictionary[i]['Best_Params']))

In [63]:
best_models_with_params

[('SGD_Classifier', {'loss': 'hinge'}),
 ('Logistic_Regression', {'penalty': None})]

In [53]:
best_models_with_params[1][0]

'Logistic_Regression'

In [58]:
best_models_with_params[i][0]

'SGD_Classifier'

In [80]:
best_estimators = {}
for i in range(len(best_models_with_params)):
    best_estimators[best_models_with_params[i][0]] = models[best_models_with_params[i][0]](**best_models_with_params[i][1])
best_estimators = list(zip(best_estimators.keys(),best_estimators.values()))
best_estimators

[('SGD_Classifier', SGDClassifier()),
 ('Logistic_Regression', LogisticRegression(penalty=None))]

In [ ]:
LogisticRegression(penalty=None).get_params

In [88]:
best_estimators[1][1].get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': None,
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [71]:
best_models_with_params

[('SGD_Classifier', {'loss': 'hinge'}),
 ('Logistic_Regression', {'penalty': None})]

In [81]:
new_dict = {}
len(new_dict)

0

In [110]:
my_dict = {'Logistic_Regression': {'Optuna': {'cost': 437200.0, 'params': {'penalty': None}}, 'HyperOpt': {'cost': 437200.0, 'params': {'penalty': None}}, 'Best_Params': {'penalty': None}, 'Best_Cost': 437200.0}}
my_dict.values()
# [value['Best_Cost'] for value in my_dict.values()]
for i in my_dict.values():
    value = i['Best_Cost']

In [111]:
value

437200.0

In [109]:
i['Best_Cost']

437200.0

In [119]:
[('KNN_Classifier', 90240.0, {'n_neighbors': 11, 'weights': 'uniform', 'algorithm': 'brute'})][0][2]

{'n_neighbors': 11, 'weights': 'uniform', 'algorithm': 'brute'}

## Config_Manager - Sorting Report Dictionary

In [ ]:
pwd

In [ ]:
from src.config.configuration_manager import ConfigurationManager
obj = ConfigurationManager()
preprocessor_config = obj.get_preprocessor_config()
os.listdir(preprocessor_config.root_dir)

In [ ]:
os.path.exists(preprocessor_config.preprocessor_path)

In [ ]:
metrics = {}
metrics['name'] = 'LogisticRegression()'
metrics['b_score'] = 9999999999.32
metrics['a_score'] = 3.23

report = {}
report['log_reg'] = metrics

metrics_a={}
metrics_a['name'] = 'SVC()'
metrics_a['b_score'] = 9934
metrics_a['a_score'] = 23534.23

report['svm'] = metrics_a

name = max(report.keys(), key = lambda k: report[k].get('a_score', 0))
report[name]

In [ ]:
report

In [ ]:
from src.utils import load_yaml,load_binary

model_dict = load_yaml(filepath = Path('artifacts\metrics\metrics.yaml'))

model = load_binary(filepath = Path('artifacts\model\model.joblib'))

preprocessor = load_binary(filepath = Path('artifacts\preprocessor\preprocessor.joblib'))

In [ ]:
df = pd.read_csv('artifacts\data\processed\stage_1_processing\preprocessed_test_data.csv')
X = df.drop(columns = 'class').iloc[:1000,:]
y = df['class'].iloc[:1000]
X_transformed = preprocessor.transform(X = X)

In [ ]:
pd.DataFrame(X_transformed).isna().sum().unique()

In [ ]:
from imblearn.combine import SMOTETomek


smote = SMOTETomek(sampling_strategy = 'minority',
                   random_state = 8,
                   n_jobs = -1)

x_smote, y_smote = smote.fit_resample(X = X_transformed,
                                      y = y)

In [ ]:
from sklearn.metrics import (balanced_accuracy_score, f1_score,
                             accuracy_score, confusion_matrix)

y_pred = model.predict(x_smote)
balanced_accuracy_score_ = float(balanced_accuracy_score(y_true=y_smote, y_pred=y_pred))
f1_score_ = float(f1_score(y_true=y_smote, y_pred=y_pred))
accuracy_score_ = float(accuracy_score(y_true=y_smote, y_pred=y_pred))
tn, fp, fn, tp = confusion_matrix(y_true=y_smote, y_pred=y_pred).ravel()
cost_ = float((10*fp)+(500*fn))

print(f'balanced_accuracy_score_: {balanced_accuracy_score_}')
print(f'accuracy_score_: {accuracy_score_}')
print(f'f1_score_: {f1_score_}')
print(f'cost_: {cost_}')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()

In [ ]:
DT.predict(x_smote)

In [ ]:
from src.utils import save_yaml
save_yaml(file=report,filepath='f:\iNeuron\Projects\Scania_Truck_Failures\sample.yaml')

In [ ]:
os.chdir('f:\\iNeuron\\Projects\\Scania_Truck_Failures')
with open (Path('.\\config\\config.yaml')) as file:
    config = yaml.safe_load(file)
print({list(config['data_ingestion_config']['test_data3'].keys())[0]:list(config['data_ingestion_config']['test_data3'].values())[0]})

In [ ]:
config['data_ingestion_config']['test_data3']

In [ ]:
any(os.listdir('../notebooks'))

In [ ]:
df = pd.read_csv('../artifacts\data\processed\stage_1_processing\preprocessed_train_data.csv')
df.shape

In [ ]:
df.drop(columns='class').describe()

In [ ]:
df1 = pd.read_csv('F:\iNeuron\Projects\Data\Scania Truck Failures\\aps_failure_training_set.csv')

In [ ]:
df1['class'] = df1['class'].map({'neg':0,'pos':1})
df1.replace('na',np.nan,inplace=True)
col_list = [i for i in df1.columns if i != 'class']
for i in col_list:
    df1[i]=df1[i].astype('float')

In [ ]:
check_df = df1.drop(columns = 'class').describe()==df.drop(columns='class').describe()
for i in check_df.columns:
    print(i,check_df[i].unique())

In [ ]:
test_df = pd.read_csv('../artifacts\data\processed\stage_1_processing\preprocessed_test_data.csv')
test_df.shape

In [ ]:
test_df1 = pd.read_csv('F:\iNeuron\Projects\Data\Scania Truck Failures\\aps_failure_test_set.csv')
test_df1['class'] = test_df1['class'].map({'neg':0,'pos':1})
test_df1.replace('na',np.nan,inplace=True)
col_list = [i for i in test_df1.columns if i != 'class']
for i in col_list:
    test_df1[i]=test_df1[i].astype('float')

In [ ]:
test_check_df = test_df.drop(columns = 'class').describe()==test_df1.drop(columns='class').describe()
for i in test_check_df.columns:
    print(i,test_check_df[i].unique())

In [ ]:
test_df1['ci_000'].describe()

In [ ]:
test_df['ci_000'].describe()

In [ ]:
df1['ci_000'].describe()

In [ ]:
df['ci_000'].describe()

In [ ]:
train_df = pd.read_csv("artifacts\data\processed\stage_2_processing\processed_train_data.csv").iloc[:1000,:]
test_df = pd.read_csv("artifacts\data\processed\stage_2_processing\processed_test_data.csv").iloc[:1000,:]

x_train = train_df.drop(columns = 'class')
y_train = train_df['class']
x_test = test_df.drop(columns = 'class')
y_test = test_df['class']

In [ ]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

In [ ]:
test_df['class'].value_counts()

In [ ]:
models = {'Logistic_Regression': LogisticRegression(), 
        'SGD_Classifier': SGDClassifier(),
        'Random Forest': RandomForestClassifier(), 
        'Ada_Boost': AdaBoostClassifier(), 
        'Grad_Boost': GradientBoostingClassifier(), 
        'Bagging_Classifier': BaggingClassifier(), 
        'ExtraTreesClassifier': ExtraTreesClassifier(), 
        'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier(), 
        'Decision_Tree_Classifier': DecisionTreeClassifier(),
        'XGB_Classifier': XGBClassifier(),
        'KNN_Classifier': KNeighborsClassifier(),
        'MLP_Classifier': MLPClassifier()
        }
report = {}
# metrics_list = ['balanced_accuracy_score','f1_score','accuracy_score']
for model_name,model in models.items():
        print(model_name)
        model.fit(x_train,y_train)
        y_pred = model.predict(x_test)
        
        metrics = {}
        metrics['balanced_accuracy_score'] = float(balanced_accuracy_score(y_true=y_test, y_pred=y_pred))

        metrics['f1_score'] = float(f1_score(y_true=y_test, y_pred=y_pred))

        metrics['accuracy_score'] = float(accuracy_score(y_true=y_test, y_pred=y_pred))

        tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
        metrics['cost'] = float((10*fp)+(500*fn))

        print(metrics,'\n')

        report[model_name] = metrics

best_model_name_by_cost = min(report.keys(), key = lambda k: report[k].get('cost', 0))
print("\nBest Model:")
print(models[best_model_name_by_cost])
print(report[best_model_name_by_cost],'\n')

In [ ]:
report

In [ ]:
report_df = pd.DataFrame(report).T

In [ ]:
report_df.sort_values(by ='cost')

In [ ]:
best_models = sorted(report.items(), key=lambda x: x[1]['cost'])[:5]
best_models

In [ ]:
best_models_names_ = [best_models[i][0] for i in range(len(best_models))]

In [ ]:
best_models_names_

In [ ]:
best_estimators = {}
for i in best_models_names_:
    best_estimators[i] = models[i]

In [ ]:
best_estimators_copy = best_estimators.copy()
best_estimators_copy

In [ ]:
best_estimators_copy = list(zip(best_estimators_copy.keys(),best_estimators_copy.values()))

In [ ]:
stacked_classifier = StackingClassifier(estimators = best_estimators_copy,
                                        final_estimator =  AdaBoostClassifier(),
                                        cv = 5,
                                        n_jobs = -1,
                                        passthrough = False,
                                        verbose = 3)

stacked_classifier.fit(x_train,y_train)
y_pred = stacked_classifier.predict(x_test)

balanced_accuracy_score_ = balanced_accuracy_score(y_true=y_test, y_pred=y_pred)
f1_score_= f1_score(y_true=y_test, y_pred=y_pred)
accuracy_score_ = accuracy_score(y_true=y_test, y_pred=y_pred)

tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
cost_ = (10*fp)+(500*fn)

print(f'balanced_accuracy_score_: {balanced_accuracy_score_}')
print(f'f1_score_: {f1_score_}')
print(f'accuracy_score_: {accuracy_score_}')
print(f'cost_: {cost_}')

In [ ]:
voting_classifier_ = VotingClassifier(estimators = best_estimators_copy,
                                      voting = "soft",
                                    weights = None,
                                    n_jobs = -1,
                                    verbose = True)
voting_classifier_.fit(x_train,y_train)
y_pred = voting_classifier_.predict(x_test)

balanced_accuracy_score_voting = balanced_accuracy_score(y_true=y_test, y_pred=y_pred)
f1_score_voting= f1_score(y_true=y_test, y_pred=y_pred)
accuracy_score_voting = accuracy_score(y_true=y_test, y_pred=y_pred)

tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
cost_voting = (10*fp)+(500*fn)

print(f'balanced_accuracy_score_: {balanced_accuracy_score_voting}')
print(f'f1_score_: {f1_score_voting}')
print(f'accuracy_score_: {accuracy_score_voting}')
print(f'cost_: {cost_voting}')

In [ ]:
df = pd.read_csv('.\\artifacts\data\processed\stage_1_processing\preprocessed_test_data.csv')

In [ ]:
df.isna().sum()

In [ ]:
df1 = pd.read_csv('F:\\iNeuron\Projects\Data\Scania Truck Failures\\aps_failure_test_set.csv')

In [ ]:
df1.isna().sum()

In [ ]:
df1.replace('na',np.nan,inplace = True)

In [ ]:
df1.drop(columns = 'class').isna().sum() == df.drop(columns = 'class').isna().sum()

In [ ]:
cols_list = df1.drop(columns = 'class').columns.tolist()
for i in cols_list:
    df1[i] = df1[i].astype('float')

In [ ]:
check_df1 = df1.drop(columns = 'class').describe() == df.drop(columns = 'class').describe()
for i in check_df1.columns:
    print(i, check_df1[i].unique())

In [ ]:
df2 = pd.read_csv('.\\artifacts\data\processed\stage_1_processing\preprocessed_train_data.csv')

In [ ]:
df2.isna().sum()

In [ ]:
df3 = pd.read_csv('F:\\iNeuron\Projects\Data\Scania Truck Failures\\aps_failure_training_set.csv')

In [ ]:
df3.replace('na',np.nan,inplace = True)
test_col_list = [i for i in df3.columns if i != 'class']
for i in test_col_list:
    df3[i]=df3[i].astype('float')
    

In [ ]:
pd.unique(df3.drop(columns = 'class').isna().sum() == df2.drop(columns = 'class').isna().sum())

In [ ]:
check_df2 = df3.drop(columns = 'class').describe() == df2.drop(columns = 'class').describe()

In [ ]:
for i in check_df2.columns:
    print(i, check_df2[i].unique())